##### Pororo를 활용한 역번역
- train_dataset에 존재하는 question 데이터를 증강하기 위해서 사용했습니다.
- question을 제외한 기존의 필드는 유지하는 조건을 가져갑니다.

----------
##### import Pororo

In [1]:
import pororo
from pororo import Pororo

/data/ephemeral/home/shark/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


----------
##### 필요 Import

In [2]:
import nltk
import torch
import random
from datasets import load_from_disk, Dataset, DatasetDict, concatenate_datasets


----------
##### 초기 설정
- nltk.download 진행 시 'punkt'가 아닌 'punkt_tab'을 입력  
※ 'punkt' 입력할 경우 mt(mt(text, src="ko", tgt="en"), src="en", tgt="ko")에서 오류 발생

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
nltk.download('punkt_tab')
mt = Pororo(task="translation", lang="multi", model='transformer.large.multi.mtpg')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


----------
##### 데이터 불러오기

In [4]:
disk = "/data/ephemeral/home/level2-mrc-nlp-12/data"

train_data = load_from_disk(disk + "/train_dataset/")
train_dataset = train_data["train"]
print(len(train_data['train'])) 

3952


----------
##### 역번역

In [5]:
def en_back_translation(text):
    return mt(mt(text, src="ko", tgt="en"), src="en", tgt="ko")

In [6]:
def ja_back_translation(text):
    return mt(mt(text, src="ko", tgt="ja"), src="ja", tgt="ko")

In [7]:
text = "케빈은 집에 갔다. 그러나 더 놀고 싶어한다."
test = ja_back_translation(text)
print(test)

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



케빈은 집에 돌아갔어. 하지만 더 놀고 싶어.


----------
##### 증강
- **돌리기 전에 en_back_translation인지, ja_back_translation인지 확인!!!!!!**

In [11]:
def augment_data(train_dataset, limit):
    augmented_examples = []
    for idx, example in enumerate(train_dataset):
        if idx >= limit:
            break
        original_question = example['question']
        
        '''
        역번역
        - 영어로 할 경우 en_back_translatiom
        - 일본어로 할 경우 ja_back_translation
        
        반드시 돌리기 전에 확인할 것!!!!!!!!!!!
        '''
        back_translated_question = ja_back_translation(original_question)           #en_back_translation(original_question)  
        
        
        # 새로운 예제 생성
        new_example = example.copy()
        new_example['question'] = back_translated_question
        
        augmented_examples.append(new_example)
    
    # 리스트를 딕셔너리로 변환하여 Dataset 생성
    aug_train_data_dict = {key: [ex[key] for ex in augmented_examples] for key in augmented_examples[0]}
    aug_train_data = Dataset.from_dict(aug_train_data_dict)
    
    return aug_train_data

**영어 역번역 데이터 :: aug_train_data**

In [9]:
#제한 개수
augmentation_limit = len(train_data['train'])       #전체

aug_train_data = augment_data(train_dataset, augmentation_limit)

**일본어 역번역 데이터 :: ja_train_data**

In [12]:
#제한 개수
augmentation_limit = len(train_data['train'])       #전체

ja_train_data = augment_data(train_dataset, augmentation_limit)

----------
##### 데이터 출력 (상위 10개)
- aug_train_data(영어) / ja_train_data(일본어) 중 택1

In [13]:
for idx, (original, augmented) in enumerate(zip(train_dataset, ja_train_data)):
    if idx >= 10:
        break
    print("[", idx, "]" + f" 기존 질문  : {original['question']}")
    print("[", idx, "]" + f" 역번역 질문: {augmented['question']}\n")

[ 0 ] 기존 질문  : 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
[ 0 ] 역번역 질문: 대통령을 포함한 미국 행정부의 견제권을 가진 국가기관은?

[ 1 ] 기존 질문  : 현대적 인사조직관리의 시발점이 된 책은?
[ 1 ] 역번역 질문: 현대인사조직관리의 시발점이 된 책은?

[ 2 ] 기존 질문  : 강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?
[ 2 ] 역번역 질문: 강희제가 1717년에 쓴 글은 누구를 위해 썼는가?

[ 3 ] 기존 질문  : 11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?
[ 3 ] 역번역 질문: 11~12세기에 제작된 본존불은 보통 어느 나라의 특징이 전파되었습니까?

[ 4 ] 기존 질문  : 명문이 적힌 유물을 구성하는 그릇의 총 개수는?
[ 4 ] 역번역 질문: 명문이 쓴 유물을 구성하는 그릇의 총수는?

[ 5 ] 기존 질문  : 카드모스의 부하들이 간 곳에는 무엇이 있었는가?
[ 5 ] 역번역 질문: 카드모스 부하가 간 곳에는 무슨 일이 있었나?

[ 6 ] 기존 질문  : 관우를 불태워 죽이려한 사람 누구인가?
[ 6 ] 역번역 질문: 관우를 불태워 죽이려고 한 사람은 누군가

[ 7 ] 기존 질문  : 참호 속에 무기와 장비를 버리고 도주한 집단은?
[ 7 ] 역번역 질문: 참호 속에 무기와 장비를 버리고 도주한 집단은?

[ 8 ] 기존 질문  : 제2차 세계 대전에 참전하여 사망한 자식은?
[ 8 ] 역번역 질문: 제2차 세계대전에 참전해서 죽은 아이는?

[ 9 ] 기존 질문  : 고려 공민왕이 처가 식구들과 아내와 함께 피신처로 삼은 마을은?
[ 9 ] 역번역 질문: 고려 공민왕이 아내의 가족과 아내와 함께 피난소로 삼은 마을은?



##### 랜덤한 30개 출력

In [14]:
random_indices = random.sample(range(len(train_dataset)), 30)

# 원래 질문과 역번역된 질문 출력 (랜덤 30개)
for idx in random_indices:
    original = train_dataset[idx]
    augmented = aug_train_data[idx]
    print("[", idx, "]" + f" 기존 질문  : {original['question']}")
    print("[", idx, "]" + f" 역번역 질문: {augmented['question']}\n")

[ 3220 ] 기존 질문  : 브리오니 협정에 따르면, 슬로베니아는 독립 선포 후 얼마동안 독립 활동을 멈춰야 하는가?
[ 3220 ] 역번역 질문: 브리오니 협정에 따르면 슬로베니아가 독립선언 이후 독립을 얼마나 중단해야 할까.

[ 2990 ] 기존 질문  : 키토 지역 토착민들이 잉카 군대에게 큰 타격을 준 곳은?
[ 2990 ] 역번역 질문: 키토에 있는 토착민들이 영국군에 어떤 큰 타격을 입는가?

[ 1745 ] 기존 질문  : 비율효과는 재화의 수요가 어떤 특징을 가지기 때문에 발생하는 효과인가?
[ 1745 ] 역번역 질문: 상품에 대한 수요가 특성을 갖고 있기 때문에 비율효과가 어떤 영향을 미치는가?

[ 515 ] 기존 질문  : 미국 기업이 선호하는 나라는 어떤 것이 저조한가요?
[ 515 ] 역번역 질문: 미국 기업들이 선호하는 나라는 무엇인가?

[ 1191 ] 기존 질문  : 비트겐슈타인이 소피야 야노프스카야를 만났던 연도는?
[ 1191 ] 역번역 질문: 비트겐슈타인은 소피다 언제 야노프스카야를 만났니?

[ 3286 ] 기존 질문  : 불리한 정보를 배제하는 이유에 대해 말한 사람은?
[ 3286 ] 역번역 질문: 당신이 왜 불리한 정보를 제외하는지 아무도 알려주지 않았나요?

[ 918 ] 기존 질문  : 호라티우스가 아테네로 공부하러 가게되었던 해는?
[ 918 ] 역번역 질문: 호라티우스는 아테네에서 어느 해 공부를 했나요?

[ 2505 ] 기존 질문  : 모투부가 1959년에 다시 방문한 곳은?
[ 2505 ] 역번역 질문: 자동차부가 1959년에 다시 방문한 것은 무엇인가?

[ 3924 ] 기존 질문  : 섭정가의 경제 기반에 큰 타격을 준 명령이 발포된 시기는?
[ 3924 ] 역번역 질문: 협상가의 경제적 기반에 부딪히는 지시 발표는 언제였을까?

[ 96 ] 기존 질문  : 소나타의 시초가 된 연습곡을 만든 사람은?
[ 96 ] 역번역 질문: 누가 소나타의 첫 연습곡을 만들었지?

[ 1325 ] 기존 질문  :

----------
##### 새로운 disk에 저장

In [17]:
combined_data = DatasetDict({'train': concatenate_datasets([train_dataset, aug_train_data, ja_train_data])})
combined_data.save_to_disk(disk + "/pororoBT_en_ja_train_data/")

Saving the dataset (1/1 shards): 100%|██████████| 11856/11856 [00:00<00:00, 240173.43 examples/s]


- 기존 + 영어 역번역 : 7904개까지 증강
- 기존 + 영어 + 일본어 : 11856개까지 증강

In [18]:
combined_data['train'][11111]

{'title': '이재명 (영화인)',
 'context': '전라남도 광주에서 출생하여 목포상업전수학교를 졸업했다. 이후 일본에 유학하여 니혼 대학 문과를 졸업한 뒤 도쿄에서 연출을 공부하고였다. 귀국한 뒤 처음 기획을 맡은 영화는 이광수의 소설을 영화화한 《무정》(1939)이다.\\n\\n1937년에 조선영화주식회사 지배인으로 입사했으며, 관제 기업인 조영에서 상당한 실세였다는 증언이 있다. 이재명은 1930년대 중반부터 영화 제작이 전문화, 분업화될 무렵에 등장한 최초의 본격적 영화 프로듀서였다.\\n\\n1940년대 국책영화 시기에 조선총독부의 협찬으로 제작된 어용영화 《흙에 산다》(1942) 등을 기획했다. 황도학회 발기인을 맡았고, 1941년 잡지 《영화평론》에서 마련한 친일 성격의 좌담회에 백철, 최인규, 허영 등과 함께 참가 한 적도 있다.\\n\\n1942년에는 조선영화주식회사가 기존의 영화사를 강제 통합하여 단일 회사로 출범했다. 이재명은 여기에 입사하여 기술과장과 촬영과장을 겸임하면서 전시체제 하의 영화계 재편 과정과 국책영화 제작에 적극적으로 협조했다. 2008년 발표된 민족문제연구소의 친일인명사전 수록예정자 명단 연극/영화 부문에 포함되었다.\\n\\n태평양 전쟁 종전 이후 조선영화주식회사의 운영권을 일본인으로부터 넘겨받은 뒤, 이를 바탕으로 결성된 조선문화건설중앙협의회의 영화 부문과 조직체인 조선영화건설본부에 참여했다. 이후 조선영화동맹의 중앙집행위원도 맡았다. 두 단체는 기본적으로 좌익 계열이었으나, 혼란기였기 때문에 여러 성향을 가진 인물들이 섞여 있었다.\\n\\n아세아영화사 대표 등을 지내며 영화제작자로 1960년대 초반까지 꾸준히 활동했다. 배우 남궁원이 집안 사정이 어려울 때 친구 아버지였던 이재명을 찾아가 도움을 청했다가 영화계에 데뷔하게 되었다는 일화가 있다.',
 'question': '무정을 기획한 인물이 태어난 도시는?',
 'id': 'mrc-0-003700',
 'answers': {'answer_start':

----------
##### 데이터 백업

In [19]:
aug_train_data.save_to_disk(disk + "/augmented_train_data/")
ja_train_data.save_to_disk(disk + "/japanese_train_data/")

Saving the dataset (1/1 shards): 100%|██████████| 3952/3952 [00:00<00:00, 222872.10 examples/s]
